<table>
<tr><td><img style="height: 150px;" src="images/geo_hydro1.jpg"></td>
<td bgcolor="#FFFFFF">
    <p style="font-size: xx-large; font-weight: 900; line-height: 100%">AG Dynamics of the Earth</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Juypter notebooks</p>
    <p style="font-size: large; color: rgba(0,0,0,0.5);">Georg Kaufmann</p>
    </td>
</tr>
</table>

# Numerical methods: Lagrangre polynomials
----
*Georg Kaufmann,
Geophysics Section,
Institute of Geological Sciences,
Freie Universität Berlin,
Germany*

In this notebook, we learn how to assemble a **Lagrange polynomial** as an approximation of
discrete data.

We first import the `python` libraries needed.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets

## Function
Next, we define the function, which we would like to examine.

In [ ]:
def pol_f(x):
    '''
    # Calculate function
    '''
    pol_f = x*np.exp(-x/5.e0)
    return pol_f

We calculate the function on the interval $[0,50]$, once with high resolution to plot
(orig), once sampled for the approximation (data):

In [ ]:
# define interval
a     = 0.e0
b     = 50.e0
## create "original" data (high resolution)
iorig = 101
xorig = np.linspace(a,b,iorig)
yorig = pol_f(xorig)
# create sample data (lower resolution)
idata = 11
xdata = np.linspace(a,b,idata)
ydata = pol_f(xdata)
# plot
plt.figure(figsize=(10.0, 6.0))
plt.rc("font",size=14)
plt.title('Lagrange Polynom interpolation',size=12)
plt.xlabel('x',size=12)
plt.ylabel('f(x)',size=12)
plt.plot(xorig,yorig,color=(0.7,0.7,0.7),linestyle='solid',linewidth=7,label='Original')
plt.plot(xdata,ydata,color='blue',marker='X',markersize=12,linewidth=0,label='Sampled')
#plt.plot(xint,yint,color='red',linestyle='solid',linewidth=2,label='Lagrange')
plt.legend(loc='upper right',shadow=True,fontsize=10)

## Lagrange basis functions
Next, we need to calculate the Lagrange basis functions:
$$
 L_{n,k}(x) = \frac{(x-x_0) (x-x_1) \dots (x-x_{k-1}) (x-x_{k+1}) \dots (x-x_n)}
                   {(x_k-x_0) (x_k-x_1) \dots (x_k-x_{k-1}) (x_k-x_{k+1}) \dots (x_k-x_n)}
$$
$$
L_{n,k}(x) =\prod\limits_{j=0, j \ne k}^{n} \frac{(x-x_j)}{(x_k-x_j)}
$$

In [ ]:
def pol_lagrange_basis(xint,xdata,n,k):
    '''
    #----------------------------------------------------------------------
    # calculate Lagrange basis function L_{n,k}
    # (c) Georg Kaufmann
    #----------------------------------------------------------------------
    '''
    P = 1
    for i in np.arange(0,n+1):
        if (i!=k):
            P = P * (xint-xdata[i]) / (xdata[k]-xdata[i])
    return P

Plot the Lagrange basis functions for a set of points in the interval $[-1,1]$:

In [ ]:
def show_lagrange_basis(n):
    # n-number of data points
    # data points
    xdata = np.linspace(-1,1,n)
    # interpolation points
    xint  = np.linspace(-1,1,10*n)
    #print(xdata)
    n=len(xdata)-1

    # plot
    plt.figure(figsize=(10,6))
    plt.xlabel('x')
    plt.ylabel('P$_{n,k}$(x)')
    for k in range(n+1):
        #print('n= ',n,'k= ',k)
        P=pol_lagrange_basis(xint,xdata,n,k)
        #print(P)
        plt.plot(xint,P,linewidth=2,label='P$_{'+str(n)+','+str(k)+'}$')
    plt.legend()
    
#lagrange_basis(4)

In [ ]:
# call interactive module
w = dict(
n=widgets.IntSlider(min=2,max=10,step=1,value=2,description='n'))

output = widgets.interactive_output(show_lagrange_basis, w)
box = widgets.VBox([widgets.HBox([*w.values()]), output])
display(box)

## Lagrange interpolation polynomial

Next, we define the Lagrange interpolation polynomial:
$$
\fbox{$
 P_n(x) =  \sum\limits_{k=0}^{n} f(x_k) L_{n,k}(x)$}
$$

In [ ]:
def pol_lagrange (xint,xdata,ydata):
    '''
    #----------------------------------------------------------------------
    # given the arrays xdata(0:n) and ydata(0:n) of length n, which tabulate
    # a function (with the xdata's in ascending order), and a coordinate xint
    # calculate the Lagrange interpolation polynomial of order n
    # (c) Georg Kaufmann
    #----------------------------------------------------------------------
    '''
    n = len(xdata)-1
    yint = 0.
    for k in np.arange(0,n+1):
        yint = yint + ydata[k] * pol_lagrange_basis(xint,xdata,n,k)
    return yint

Finally, interpolate our re-sampled function with the Lagrange interpolation polynomial.

In [ ]:
def show_lagrange_interpol(n):
    # create sample data (lower resolution)
    idata = n
    xdata = np.linspace(a,b,idata)
    ydata = pol_f(xdata)
    iint  = 41
    xint  = np.linspace(a,b,iint)
    yint  = np.zeros(iint)
    for i in np.arange(0,len(xint)):
        yint[i]  = pol_lagrange(xint[i],xdata,ydata)
    # plot
    plt.figure(figsize=(10.0, 6.0))
    plt.rc("font",size=14)
    plt.title('Lagrange Polynom interpolation P$_{'+str(idata-1)+'}$(x)',size=12)
    plt.xlim([a,b])
    plt.ylim([-0.5,2])
    plt.xlabel('x',size=12)
    plt.ylabel('f(x)',size=12)
    plt.plot(xorig,yorig,color=(0.7,0.7,0.7),linestyle='solid',linewidth=7,label='Original')
    plt.plot(xdata,ydata,color='blue',marker='X',markersize=12,linewidth=0,label='Sampled')
    plt.plot(xint,yint,color='red',linestyle='solid',linewidth=2,label='Lagrange')
    plt.legend(loc='upper right',shadow=True,fontsize=10)

In [ ]:
# call interactive module
w = dict(
n=widgets.IntSlider(min=2,max=15,step=1,value=2,description='n'))

output = widgets.interactive_output(show_lagrange_interpol, w)
box = widgets.VBox([widgets.HBox([*w.values()]), output])
display(box)

## And back to the example $f(x)={{1}\over{x}}$ example

In [ ]:
from scipy.interpolate import lagrange
from numpy.polynomial.polynomial import Polynomial
# define function
def f(x):
    f = 1/x
    return f
# define analytical solution
xorig = np.linspace(0.1,6,61)
yorig = f(xorig)
# define sampling points and use lagrange from python
x = np.array([2.0,2.5,4.0])
y = f(x)
poly = lagrange(x, y)
coeff = Polynomial(poly).coef
# plot
plt.figure(figsize=(10,6))
plt.xlabel('x')
plt.ylabel('f(x)')
plt.ylim([0,6])
plt.plot(xorig,yorig,linewidth=8,color='gray',label='f(x)=1/x')
plt.plot(xorig,coeff[0]*xorig**2+coeff[1]*xorig+coeff[2],linewidth=3,color='red',label='P$_2$(x)')
plt.legend()

[next >](Numerics_lab04_splines.ipynb)